## Colab

In [1]:
### Pour utiliser sur google colab

import os

from google.colab import drive
drive.mount('drive')

!mkdir -p drive -v
#!google-drive-ocamlfuse drive

cwd = os.getcwd()

#### Changez le chemin ci-dessous vers votre repertoire dans votre googledrive
##############################################################################

monchemin = 'drive/My Drive/M2-IAAA/TAL/Projet/'

dir_path  = os.path.join(cwd, monchemin)
dirs = os.listdir(dir_path)
os.chdir(dir_path)

###  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [2]:
!ls

Arc-eager.ipynb  f1_fr	f2_fr  UD_French-GSD


## Word

In [0]:
class Word:
    def __init__(self):
        self.featDic = {}

    def getFeat(self, featName):
        if(not featName in self.featDic):
            print('WARNING : feat', featName, 'does not exist')
            return None
        else:
            return self.featDic[featName]

    def setFeat(self, featName, featValue):
        self.featDic[featName] = featValue

    def affiche(self, mcd):
        for elt in mcd:
            feat, status = elt
            print(self.getFeat(feat), '\t', end='')
        print('')

        
    @staticmethod
    def fakeWord(mcd):
        w =Word()
        for elt in mcd:
            feat, status = elt
            w.setFeat(feat, 'ROOT')
        return w

    @staticmethod
    def invalidGov():
        return 123456789

    @staticmethod
    def invalidLabel():
        return ''


## WordBuffer

In [0]:


class WordBuffer:
        def __init__(self, mcd=None):
                self.currentIndex = 0
                self.array = []
                self.length = 0
                self.mcd = mcd
                
        def addWord(self, w):
                self.array.append(w)
                self.length += 1

        def affiche(self, mcd):
                for w in self.array:
                        w.affiche(mcd)

        def getLength(self):
                return self.length
        
        def getCurrentIndex(self):
                return self.currentIndex

        def getWord(self, index):
                return self.array[index]

        def getCurrentWord(self):
                return self.getWord(self.currentIndex)
            
        def nextSentence(self):
                sentence = []
                sentence.append(Word.fakeWord(self.mcd))
                if self.currentIndex == self.length - 1 :
                        return False
                while self.currentIndex < self.length :
                        sentence.append(self.getCurrentWord())
#                        self.getCurrentWord().affiche(self.mcd)
                        if int(self.getCurrentWord().getFeat('EOS')) == 1 :
                                self.currentIndex += 1
                                return sentence
                        else:
                                self.currentIndex += 1
                
        def readFromMcfFile(self, mcfFilename):
                try:
                        mcfFile = open(mcfFilename, encoding='utf-8')
                except IOError:
                        print(mcfFilename, " : ce fichier n'existe pas")
                        exit(1)
                tokens = []
                for ligne in mcfFile:
                        tokens = ligne.split()
                        w = Word()
                        for i in range(0, len(tokens)):
                                if(self.mcd[i][0] == 'GOV'):
                                        w.setFeat(self.mcd[i][0], tokens[i])
                                        w.setFeat('GOVABS', str(self.length + int(tokens[i]))) # compute absolute index of governor
                                else:
                                        w.setFeat(self.mcd[i][0], tokens[i])
                        self.addWord(w)
                mcfFile.close();
                
        def readFromConlluFile(self, conlluFilename):
                conlluFile = open(conlluFilename, encoding='utf-8')
                
                tokens = []
                for ligne in conlluFile:
#                        print(ligne)
                        if ligne[0] == '\n' :
                                self.getWord(self.currentIndex - 1).setFeat('EOS', '1')
                                next
                        elif ligne[0] == '#' :
#                                print("commentaire")
                                next
                        else :
#                                1	Je	il	PRON	_	Number=Sing|Person=1|PronType=Prs	2	nsubj	_	_
#featModel = (('B', 0, 'POS'),('S', 0, 'POS'), ('B', 0, 'GOV'), ('S', 0, 'GOV'), ('B', -1, 'POS'), ('B', 1, 'POS'))
                                tokens = ligne.split("\t")
                                if '-' not in tokens[0]:
                                        w = Word()
                                        for i in range(0, len(tokens)):
                                                if i == 0 :
                                                        w.setFeat('INDEX', tokens[i])
                                                if i == 1 :
                                                        w.setFeat('FORM', tokens[i])
                                                if i == 2 :
                                                        w.setFeat('LEMMA', tokens[i])
                                                if i == 3 :
                                                        w.setFeat('POS', tokens[i])
                                                if i == 4 :
                                                        w.setFeat('X1', tokens[i])
                                                if i == 5 :
                                                        w.setFeat('MORPHO', tokens[i])
                                                if i == 6 :
                                                        w.setFeat('GOV', tokens[i])
                                                if i == 7 :
                                                        w.setFeat('LABEL', tokens[i])
                                                if i == 8 :
                                                        w.setFeat('X2', tokens[i])
                                                if i == 9 :
                                                        w.setFeat('X3', tokens[i])
                                        w.setFeat('EOS', '0')
                                        self.addWord(w)
                conlluFile.close();
                
        def end(self):
                if(self.getCurrentIndex() >= self.getLength()):
                        return True
                else:
                        return False

## Dicos

In [0]:
class Dicos:
        def __init__(self, mcd=False, fileName=False, verbose=False):
                self.content = {}
                self.locked = False
                if mcd :
                        for elt in mcd :
                                name, status = elt;
                                if(status == 'SYM') : self.content[name] = ['NULL', 'ROOT']
                elif fileName :
                        try:
                                dicoFile = open(fileName, encoding='utf-8')
                        except IOError:
                                print(fileName, 'does not exist')
                                exit(1)
                        for ligne in dicoFile:
                                if ligne[0] == '#' and ligne[1] == '#' :
                                        currentDico = ligne[2:-1]
                                        self.content[currentDico] = []
                                        if(verbose): print('in module', __name__, 'create dico', currentDico)
                                else:
                                        value = ligne[:-1]
                                        if not value in self.content[currentDico] :
                                                self.content[currentDico].append(value)
                                                if(verbose): print('in module', __name__, 'adding entry', value, 'to', currentDico)
                        dicoFile.close()
                        self.lock()


        def populateFromMcfFile(self, mcfFilename, mcd, verbose=False):
                try:
                        mcfFile = open(mcfFilename, encoding='utf-8')
                except IOError:
                        print('cannot open', mcfFilename)
                        exit(1)
                        tokens = []
                for ligne in mcfFile:
                        tokens = ligne.split()
                        for i in range(0, len(tokens)):
                                if mcd[i][1] == 'SYM' :
                                        if not tokens[i] in self.content[mcd[i][0]] :
                                                self.content[mcd[i][0]].append(tokens[i])
                                                if(verbose): print('in module:', __name__, 'adding value ', tokens[i], 'to dico', mcd[i][0]) 
                mcfFile.close();
                for e in self.content:
                        print('DICO', e, ':\t', len(self.content[e]), 'entries')
                                                        
        def populateFromConlluFile(self, conlluFilename, verbose=False):
                mots_set = set()
                mots = []
                pdd_set = set()
                pdd = []
                lemma_set = set()
                lemma = []
                morpho_set = set()
                morpho = []
                try:
                        conlluFile = open(conlluFilename, encoding='utf-8')
                except IOError:
                        print('cannot open', conlluFilename)
                        exit(1)
                mcd = (('INDEX', 'INT'), ('FORM', 'INT'), ('LEMMA', 'INT'), ('POS', 'SYM'), ('X1', 'INT'), ('MORPHO', 'INT'), ('GOV', 'SYM'), ('LABEL', 'SYM'), ('X2', 'SYM'), ('X3', 'SYM'))
                tokens = []
                for ligne in conlluFile:
                        if ligne[0] != '\n' and ligne[0] != '#' :
                                tokens = ligne.split("\t")
                                for i in range(0, len(tokens)):

                                        if tokens[1] not in mots_set:  
                                          mots_set.add(tokens[1])
                                          mots.append(tokens[1])
                                        if tokens[3] not in pdd_set:  
                                          pdd_set.add(tokens[3])
                                          pdd.append(tokens[3])
                                        if tokens[2] not in lemma_set:  
                                          lemma_set.add(tokens[2])
                                          lemma.append(tokens[2])
                                        if tokens[5] not in morpho_set:  
                                          morpho_set.add(tokens[5])
                                          morpho.append(tokens[5])
                                          
                                        if mcd[i][1] == 'SYM' :
                                                if not tokens[i] in self.content[mcd[i][0]] :
                                                        self.content[mcd[i][0]].append(tokens[i])
                                                        if(verbose): print('in module:', __name__, 'adding value ', tokens[i], 'to dico', mcd[i][0]) 
                conlluFile.close();
                #for e in self.content:
                       # print('DICO', e, ':\t', len(self.content[e]), 'entries')
                return mots,pdd,lemma,morpho
                                                        
        def lock(self):
                self.locked = True
                for key in self.content.keys():
                        self.content[key] = tuple(self.content[key])

        def print(self):
                for dico in self.content.keys():
                        print(dico, self.content[dico])

        def printToFile(self, filename):
            try:
                dicoFile = open(filename, 'w', encoding='utf-8')
            except IOError:
                print('cannot open', filename)
                exit(1)
            for dico in self.content.keys():
                dicoFile.write('##')
                dicoFile.write(dico)
                dicoFile.write('\n')
                for elt in self.content[dico]:
                    dicoFile.write(elt)
                    dicoFile.write('\n')
            dicoFile.close()

        def getCode(self, dicoName, symbol, verbose=False) :
                if(verbose) : print('in module ', __name__, 'getCode(', dicoName, ',', symbol, ')')
                if not self.locked :
                        print('Dicos must be locked before they can be accessed')
                        return False
                if not dicoName in self.content :
                        print('no such dico as', dicoName)
                        return False
#                print('dicoName =', dicoName, 'symbol =', symbol)
                return self.content[dicoName].index(symbol)

        def getSymbol(self, dicoName, code) :
                if not self.locked :
                        print('Dicos must be locked before they can be accessed')
                        return False
                if not dicoName in self.content :
                        print('no such dico as', dicoName)
                        return False
                return self.content[dicoName][code]

        def add(self, dicoName, symbol) :
                if not dicoName in self.content :
                        self.content[dicoName] = {symbol}
                else:
                        self.content[dicoName].add(symbol)



## create_dicos

In [0]:
import sys

def getUniqueWords(allWords) :
    uniqueWords = [] 
    for i in allWords:
        if not i in uniqueWords:
            uniqueWords.append(i)
    return uniqueWords

if len(sys.argv) < 3 :
    print('usage:', sys.argv[0], 'conllu_file (input) dico_file (output)')
    exit(1)


mcd = (('INDEX', 'INT'), ('FORM', 'INT'), ('LEMMA', 'INT'), ('POS', 'SYM'), ('X1', 'INT'), ('MORPHO', 'INT'), ('GOV', 'SYM'), ('LABEL', 'SYM'), ('X2', 'SYM'), ('X3', 'SYM'))


print('populating dicos from file ', "UD_French-GSD/fr_gsd-ud-train.conllu")
dicos = Dicos(mcd)
mots,pdd = dicos.populateFromConlluFile("UD_French-GSD/fr_gsd-ud-train.conllu", verbose=False)
dicos.lock()
mots += ["Root"]
pdd += ["Root"]
#print('saving dicos in file ', sys.argv[2])
#dicos.printToFile(sys.argv[2])



populating dicos from file  UD_French-GSD/fr_gsd-ud-train.conllu


ValueError: ignored

In [0]:
def create_vocab(filename):
  mcd = (('INDEX', 'INT'), ('FORM', 'INT'), ('LEMMA', 'INT'), ('POS', 'SYM'), ('X1', 'INT'), ('MORPHO', 'INT'), ('GOV', 'SYM'), ('LABEL', 'SYM'), ('X2', 'SYM'), ('X3', 'SYM'))


  dicos = Dicos(mcd)
  mots,pdd,lemma,morpho = dicos.populateFromConlluFile(filename, verbose=False)
  dicos.lock()
  mots += ["Root"] 
  pdd += ["Root"] + ["N..U..L..L"]
  lemma += ["Root"]
  morpho += ["Root"]
  
  return mots,pdd,lemma,morpho

## read_conllu

In [57]:

import sys



mcd =(('INDEX', 'INT'), ('FORM', 'INT'), ('LEMMA', 'INT'), ('POS', 'SYM'), ('X1', 'INT'), ('MORPHO', 'INT'), ('GOV', 'SYM'), ('LABEL', 'SYM'), ('X2', 'SYM'), ('X3', 'SYM'))

wb = WordBuffer(mcd);
wb.readFromConlluFile("UD_French-GSD/fr_gsd-ud-train.conllu");

#wb.affiche(mcd)

def printSentence(sentence, mcd):
    for i in range(0, len(sentence)):
        sentence[i].affiche(mcd)


sentence = wb.nextSentence()
sentNb = 1
while sentence :
    print('sentence', sentNb)
    printSentence(sentence, mcd)
    sentNb += 1
    sentence = wb.nextSentence()


sentence 1
ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	ROOT 	
1 	Les 	le 	DET 	_ 	Definite=Def|Gender=Fem|Number=Plur|PronType=Art 	2 	det 	_ 	_
 	
2 	commotions 	commotion 	NOUN 	_ 	Gender=Fem|Number=Plur 	5 	nsubj 	_ 	_
 	
3 	cérébrales 	cérébral 	ADJ 	_ 	Gender=Fem|Number=Plur 	2 	amod 	_ 	_
 	
4 	sont 	être 	AUX 	_ 	Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin 	5 	aux 	_ 	_
 	
5 	devenu 	devenir 	VERB 	_ 	Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part 	0 	root 	_ 	Correct=devenues
 	
6 	si 	si 	ADV 	_ 	_ 	7 	advmod 	_ 	_
 	
7 	courantes 	courant 	ADJ 	_ 	Gender=Fem|Number=Plur 	5 	xcomp 	_ 	_
 	
8 	dans 	dans 	ADP 	_ 	_ 	10 	case 	_ 	_
 	
9 	ce 	ce 	DET 	_ 	Gender=Masc|Number=Sing|PronType=Dem 	10 	det 	_ 	_
 	
10 	sport 	sport 	NOUN 	_ 	Gender=Masc|Number=Sing 	7 	obl:mod 	_ 	_
 	
11 	qu' 	que 	SCONJ 	_ 	_ 	14 	mark 	_ 	SpaceAfter=No
 	
12 	on 	on 	PRON 	_ 	Gender=Masc|Number=Sing|Person=3 	14 	nsubj 	_ 	_
 	
13 	les 	le 	PRON 	_ 	Number=Plur|Person=3|PronType

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



 	NOUN 	_ 	Gender=Masc|Number=Plur 	6 	conj 	_ 	_
 	
10 	( 	( 	PUNCT 	_ 	_ 	13 	punct 	_ 	SpaceAfter=No
 	
11 	dont 	dont 	PRON 	_ 	PronType=Rel 	13 	nmod 	_ 	_
 	
12 	trois 	trois 	NUM 	_ 	_ 	13 	nummod 	_ 	_
 	
13 	enfants 	enfant 	NOUN 	_ 	Gender=Masc|Number=Plur 	9 	appos 	_ 	_
 	
14 	Victor 	Victor 	PROPN 	_ 	_ 	13 	appos 	_ 	_
 	
15 	: 	: 	PUNCT 	_ 	_ 	13 	punct 	_ 	_
 	
16 	Jean-Christophe 	Jean-Christophe 	PROPN 	_Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-57-bb0ab605b156>", line 22, in <module>
    printSentence(sentence, mcd)
  File "<ipython-input-57-bb0ab605b156>", line 15, in printSentence
    sentence[i].affiche(mcd)
  File "<ipython-input-3-f5ece7e0fc48>", line 18, in affiche
    print(self.getFeat(feat), '\t', end='')
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py", line 350, i

KeyboardInterrupt: ignored

# Get data

In [0]:
import copy
def get_data(filename,feature):
  try:
    conlluFile = open(filename, encoding='utf-8')
  except IOError:
    print('cannot open', conlluFilename)
    exit(1)

  phrase = [] 
  X = []
  Y = []
  
  go = 0
  fa = 0

  for ligne in conlluFile:
    if ligne[0] == '\n': #Nouvelle phrase
      pa = copy.copy(phrase)
      couple = verif_arcs(phrase,feature)
    
      arcs,x,y = parser(phrase,feature)
      X += x
      Y += y
      if ( not verif_couple(couple,arcs) ):
        fa += 1
        print()
        print(pa)
        print(arcs)
        print(couple)
        #sys.exit("Error message")
      else :
        go += 1
        #print()
        #print(pa)
        #print(arcs)
        #print(couple)
        #print(couple)
        #print(arcs)
        #sys.exit("Error message")
   
      phrase = []
    if ligne[0] != '\n' and ligne[0] != '#' :
      tokens = ligne.split("\t")
      #for i in range(0, len(tokens)):
      #print(tokens[6])
      #print(tokens)
      if ( "-" not in tokens[0]):
        if ( feature == "f1"):
        # FORM , POS , GOV , LABEL
          phrase.append([tokens[1],tokens[3],tokens[6],tokens[7]])
        elif ( feature == "f2" or feature == "f3"):
        # FORM , POS , GOV , LABEL, LEMMA , MORPHO
          phrase.append([tokens[1],tokens[3],tokens[6],tokens[7],tokens[2],tokens[5]])
       
  print("True  : ",go , " / False : " , fa)
  #sys.exit("Error message")
  return X,Y

def get_unique(X,Y):
  b = []
  for i in range(len(X)-1):
    b.append(X[i] + [Y[i]])


  c = np.unique(b, axis=0)
  X = []
  Y = []

  for j in range(len(c)-1):

    X.append([c[j][0],c[j][1],c[j][2],c[j][3],c[j][4]])
    Y.append(c[j][5])
    
  return X,Y





In [0]:
import sys
def verif_arcs(phrase,features):
  couple = []
  for i in range(len(phrase)):
    
    if ( phrase[i][3] == "root" ):
      if (features == "f1"):
        couple.append((phrase[i],["Root","Root",0,"Root"]))
      else : 
        couple.append((phrase[i],["Root","Root",0,"Root","Root","Root"]))
    else :
      couple.append((phrase[i],phrase[int(phrase[i][2])-1]))
 
  return couple

def verif_couple(couple,arcs):

  for i in range(len(arcs)):
    if (arcs[i] not in couple):
      return False
  
  return True
     
  

# Arc eager

Buffer , decaller l'indice au lieu de supprimer

In [0]:
from random import randint
import numpy as np





def left_arc(stack,buffer,arcs,tab_head,index_phrase,needs):
  arcs.append((buffer[0],stack[len(stack)-1]))
  if ( str(index_phrase.index(stack[len(stack)-1][0]))  not in needs): # Si le dernier mot du stack a deja sont head / stack[len(stack)-1]  in tab_head and
    stack.pop(len(stack)-1)
  else:
    word = buffer.pop(0) # Retire le premier mot du buffer 
    stack.append(word)
  return stack,buffer,arcs
  
def right_arc(stack,buffer,arcs,tab_head,index_phrase,needs):
  arcs.append((stack[len(stack)-1],buffer[0]))
  if ( buffer[0] in tab_head and str(index_phrase.index(buffer[0][0]))  not in needs): # Si le premier mot du buffer à deja sont head  / buffer[0] in tab_head and
    word = buffer.pop(0)
    stack.append(word)
  else :
     stack.pop(len(stack)-1)
  return stack,buffer,arcs
  
def shift(stack,buffer):
  word = buffer.pop(0)
  stack.append(word)
  return stack , buffer
  
def reduce(stack):
  stack.pop(len(stack)-1)
  return stack

def oracle(w1,w2,index_phrase,phrase,dist,tab_head,needs):

  #print(w2[2])
  if ( int(w1[2]) == phrase.index(w2) ):
    if ( str(index_phrase.index(w2[0])) in needs ): 
      needs.pop(needs.index(str(index_phrase.index(w2[0]))))
    tab_head.append(w1)
    return "RIGHT_"+str(w1[3]),2,tab_head,needs
  
  elif ( int(w2[2]) == phrase.index(w1)):
    if ( str(index_phrase.index(w1[0])) in needs ):
      needs.pop(needs.index(str(index_phrase.index(w1[0]))))
    tab_head.append(w2)
    return "LEFT_"+str(w2[3]),1,tab_head,needs
  
  elif (w1 in tab_head and str(index_phrase.index(w1[0])) not in needs) : 
    return "REDUCE",4,tab_head,needs
  
  else : 
    return "SHIFT",3,tab_head,needs
  

def listing(sentence):
  res = []
  for i in range(len(sentence)):
    res.append(i + 1)
  return res

def print_result(res,sentence):
  sentence = ["root"] + sentence
  for i in range(len(res)):
    w1 , w2 = res[i]
    print(sentence[w1],' -> ', sentence[w2])
    
def get_need(phrase):
  need = []
  [need.append(i[2]) for i in phrase]
  return need
    
    
def parser(phrase,feature):
  tab_head = []
  buffer = phrase
  needs = get_need(phrase)
  if (feature == "f1"):
    root = ["Root","Root",0,"Root"]
  elif (feature == "f2" or feature == "f3" ):
    root = ["Root","Root",0,"Root","Root","Root"]
  phrase = [root] + phrase
  index_phrase = [i[0] for i in phrase]
  arcs = []
  stack = []
  stack.append(root) ## root
  
  X = []
  Y = []
  
  stack_done =[]
  buffer_done =[]
  
  while ( (len(buffer) != 0) & (len(stack) != 0)):
    #print()
    #print("buffer : ", [i[0] for i in buffer])
    #print("stack : ", [i[0] for i in stack])
    #print(needs)
    
    if( stack[len(stack)-1] == root ):
      dist = 0 -phrase.index(buffer[0])
    elif (buffer[0] == root ):
      dist = phrase.index(stack[len(stack)-1])
    else :
      dist = phrase.index(stack[len(stack)-1])-phrase.index(buffer[0])
    
    if dist > 7 : dist = 7
    if dist < -7 : dist = -7
 
    if ( feature == "f1"):
      X.append([stack[len(stack)-1][0], # FORM mot 1
                buffer[0][0],           # POS mot 1 
                stack[len(stack)-1][1], # FORM mot 2
                buffer[0][1],dist])     # FORM mot 2
    if ( feature == "f2"):
      
      if ( len(stack_done) >= 1 ):
        stack_before_pos = stack_done[len(stack_done)-1]
      else :
        stack_before_pos = "N..U..L..L"
        
      stack_done.append(stack[len(stack)-1][1])
        
      if ( len(buffer_done) >= 1 ):
        buffer_before_pos = buffer_done[len(buffer_done)-1]
      else :
        buffer_before_pos = "N..U..L..L"
        
      buffer_done.append(buffer[0][1])
      
      if ( len(buffer) >= 2 ):
        buffer_after_pos = buffer[1][1]
      else :
        buffer_after_pos = "N..U..L..L"
        
      X.append([stack[len(stack)-1][0], # FORM mot 1
                stack[len(stack)-1][1], # POS mot 1
                stack[len(stack)-1][4], # LEMMA mot 1
                stack[len(stack)-1][5], # MORPHO mot 1
                stack_before_pos,       # POS mot -1
                buffer[0][0], # FORM mot 2
                buffer[0][1], # POS mot 2
                buffer[0][4], # LEMMA mot 2
                buffer[0][5], # MORPHO mot 2
                buffer_before_pos,
                buffer_after_pos,
                dist])
    if ( feature == "f3"):
      
      if ( len(stack) >= 2 ):
        stack_after_pos = stack[len(stack)-2][1]
      else :
        stack_after_pos = "N..U..L..L"
        
      if ( len(buffer_done) >= 1 ):
        buffer_before_pos = buffer_done[len(buffer_done)-1]
      else :
        buffer_before_pos = "N..U..L..L"

      buffer_done.append(buffer[0][1])

      if ( len(buffer) >= 2 ):
        buffer_after_pos = buffer[1][1]
      else :
        buffer_after_pos = "N..U..L..L"


      X.append([stack[len(stack)-1][0], # FORM mot 1
          stack[len(stack)-1][1], # POS mot 1
          stack[len(stack)-1][4], # LEMMA mot 1
          stack[len(stack)-1][5], # MORPHO mot 1
          stack_after_pos,       # POS mot 1
          buffer[0][0], # FORM mot 2
          buffer[0][1], # POS mot 2
          buffer[0][4], # LEMMA mot 2
          buffer[0][5], # MORPHO mot 2
          buffer_before_pos,
          buffer_after_pos,
          dist])

      

     
   
    Y_actu, gold,tab_head,needs = oracle(stack[len(stack)-1],buffer[0],index_phrase,phrase,dist,tab_head,needs)
  
    Y.append(Y_actu)
    
    if ( gold == 1 ):
      stack,buffer,arcs = left_arc(stack,buffer,arcs,tab_head,index_phrase,needs)
    elif (gold == 2):
      stack,buffer,arcs = right_arc(stack,buffer,arcs,tab_head,index_phrase,needs)
    elif (gold == 3):
      stack,buffer = shift(stack,buffer)
    elif (gold == 4):
      stack = reduce(stack)
      
  return arcs,X,Y



    

In [0]:
def convert_x_index(x,vocab,pdd,lemma,morpho,feature):
  if (feature == "f1"):
    for i in range(len(x)):

      x[i][0] = vocab.index(x[i][0])
      x[i][1] = vocab.index(x[i][1])
      x[i][2] = pdd.index(x[i][2])
      x[i][3] = pdd.index(x[i][3])
      x[i][4] = int(x[i][4])
  elif (feature == "f2" or feature == "f3"):
    for i in range(len(x)):
      x[i][0] = vocab.index(x[i][0])   
      x[i][1] = pdd.index(x[i][1])
      x[i][2] = lemma.index(x[i][2])
      x[i][3] = morpho.index(x[i][3])
      x[i][4] = pdd.index(x[i][4])
      x[i][5] = vocab.index(x[i][5])
      x[i][6] = pdd.index(x[i][6])
      x[i][7] = lemma.index(x[i][7])
      x[i][8] = morpho.index(x[i][8])
      x[i][9] = pdd.index(x[i][9])
      x[i][10] = pdd.index(x[i][10])
      x[i][11] = int(x[i][11])
  return x

def convert_y_index(y,vocab):
  for i in range(len(y)):
    y[i] = vocab.index(y[i])
    
  return y

In [0]:
def get_all_data(filename,feature="f1"):
  # Input : conll file
  # Output : X  = [ mot1 , mot2 , pdd1 , pdd2, dist]
  #          Y  = [Liaison mots1 - mot2]
  #          mots = vocabulaires des mots
  #          pdd  = vocabulaires des parties de discours
  #          lemma = vocabulaire des lemmes
  #          morpho = vocabulaire des morphos
  #          vocab_y = vocabulaire des liaisons
  mots, pdd,lemma,morpho = create_vocab(filename)
  X,Y = get_data(filename,feature)
  vocab_y = np.unique(Y)
  X_unique_index = convert_x_index(X,mots,pdd,lemma,morpho,feature)
  Y_unique_index = convert_y_index(Y,vocab_y.tolist())
  
  if (feature == "f1"):
    return X_unique_index, Y_unique_index , mots, pdd, vocab_y
  if  (feature == "f2" or feature == "f3"):
    return X_unique_index, Y_unique_index , mots, pdd, lemma, morpho, vocab_y

In [128]:
import pickle
feat = "f3"
if ( feat == "f1"):
  X , Y , vocab_mots , vocab_pdd , vocab_liaisons = get_all_data("UD_French-GSD/fr_gsd-ud-train.conllu",feature=feat)
  dico = { "X" : X , "Y":Y , "vocab_mots":vocab_mots , "vocab_pdd":vocab_pdd , "vocab_liaisons":vocab_liaisons}
  f = open("f1_fr","wb")
  pickle.dump(dico,f)
  f.close()
elif (feat =="f2"):
  X , Y , vocab_mots , vocab_pdd , vocab_lemma, vocab_morpho, vocab_liaisons = get_all_data("UD_French-GSD/fr_gsd-ud-train.conllu",feature=feat)
  dico = { "X" : X , "Y":Y , "vocab_mots":vocab_mots , "vocab_pdd":vocab_pdd , "vocab_liaisons":vocab_liaisons}
  f = open("f2_fr","wb")
  pickle.dump(dico,f)
  f.close()
elif (feat =="f3"):
  X , Y , vocab_mots , vocab_pdd , vocab_lemma, vocab_morpho, vocab_liaisons = get_all_data("UD_French-GSD/fr_gsd-ud-train.conllu",feature=feat)
  dico = { "X" : X , "Y":Y , "vocab_mots":vocab_mots , "vocab_pdd":vocab_pdd , "vocab_liaisons":vocab_liaisons}
  f = open("f3_fr","wb")
  pickle.dump(dico,f)
  f.close()

True  :  14554  / False :  0


In [123]:
print(len(X),len(Y),len(vocab_mots),len(vocab_pdd),len(vocab_morpho))
print(X[5000])


215907 215907 42278 20 143
['Carinae', 'PROPN', 'Carinae', '_', ['Root', 'Root', 0, 'Root', 'Root', 'Root'], '(', 'PUNCT', '(', '_', 'PROPN', 'NUM', -1]


Sortir X et Y 
Sortir liste de mot ( vocab )

mot1 = stack[0] 
mot2 = buffer[0]

X = [ mot1 , mot2 , pdd1 , pdd2, dist]
mot1 = vocab_mot[mot1]
pdd = vocab_pdd

Y = vocab_liaison

si dist > 7

voc_liaison[0 = shift , 1 = reduce , left_1,  right_1 , left_2 ...  ]
